In [1]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from sklearn.preprocessing import MultiLabelBinarizer
# from tensorflow.keras.layers.merge import Concatenate
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
toxic_comments = pd.read_csv('/content/drive/MyDrive/NLP/1613373921-5e748a2d5fc288e9f69c5f86.csv')

In [4]:
toxic_comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
filter = toxic_comments["comment_text"] != ""
toxic_comments = toxic_comments[filter]
toxic_comments = toxic_comments.dropna()

In [6]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [7]:
X = []
sentences = list(toxic_comments["comment_text"])
for sen in sentences:
    X.append(preprocess_text(sen))

In [8]:
#multi-class classification
toxic = toxic_comments[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] > 0

In [9]:
print(toxic.shape)

(159571, 6)


In [10]:
from nltk.corpus import stopwords

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
my_stopwords = stopwords.words('english')

In [13]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/MyDrive/NLP/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

### Vectorize the comments

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = my_stopwords)

##Different vecctorization methods were implemeted and the results were observed##

#vectorizer = CountVectorizer(analyzer='word',       
                    #         min_df=10,                        # minimum reqd occurences of a word 
                    #         stop_words='english',             # remove stop words
                    #         lowercase=True,                   # convert all words to lowercase
                    #         token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                    #          max_features=50000,              #  max number of uniq words
                    #                             )
#vectorizer = CountVectorizer(analyzer='word',       
                    #        min_df=10,                        # minimum reqd occurences of a word 
                    #         stop_words='english',             # remove stop words
                    #         lowercase=True,                   # convert all words to lowercase
                    #         token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                    #         binary=True #Returns a Binary Vector
                    #         max_features=50000,             # max number of uniq words
                    #                               )
bag_of_words = vectorizer.fit_transform(X)

In [15]:
print("Sparsity of bag_of_words:",
      len(bag_of_words.data)/(bag_of_words.shape[0]*bag_of_words.shape[1])*100)

Sparsity of bag_of_words: 0.016165423205353118


In [16]:
list(vectorizer.vocabulary_.keys())

['explanation',
 'edits',
 'made',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'reverted',
 'vandalisms',
 'closure',
 'gas',
 'voted',
 'new',
 'york',
 'dolls',
 'fac',
 'please',
 'remove',
 'template',
 'talk',
 'page',
 'since',
 'retired',
 'aww',
 'matches',
 'background',
 'colour',
 'seemingly',
 'stuck',
 'thanks',
 'january',
 'utc',
 'hey',
 'man',
 'really',
 'trying',
 'edit',
 'war',
 'guy',
 'constantly',
 'removing',
 'relevant',
 'information',
 'talking',
 'instead',
 'seems',
 'care',
 'formatting',
 'actual',
 'info',
 'make',
 'real',
 'suggestions',
 'improvement',
 'wondered',
 'section',
 'statistics',
 'later',
 'subsection',
 'types',
 'accidents',
 'think',
 'references',
 'may',
 'need',
 'tidying',
 'exact',
 'format',
 'ie',
 'date',
 'etc',
 'one',
 'else',
 'first',
 'preferences',
 'style',
 'want',
 'let',
 'know',
 'appears',
 'backlog',
 'articles',
 'review',
 'guess',
 'delay',
 'reviewer',
 'turns',
 'listed',
 'form',
 'eg',
 'wikipedia',
 

In [17]:
len(list(vectorizer.vocabulary_.keys()))

168645

In [18]:
embeds = []
zeros_embed = 100*[0]
for key,val in sorted(vectorizer.vocabulary_.items(), key = lambda kv:(kv[1], kv[0])):
# vectorizer.vocabulary_.keys():
    if key in embeddings_dictionary:
        embeds.append(embeddings_dictionary[key])
    else:
        embeds.append(zeros_embed)
    if val%1000 == 0:
        print(val,key)


0 aa
1000 accorta
2000 adminnaizsi
3000 aggrevation
4000 albo
5000 altor
6000 andjapan
7000 antiquary
8000 aquainted
9000 arrogans
10000 associazioni
11000 austrolestes
12000 baceline
13000 bannock
14000 bd
15000 benegiciary
16000 bidhonene
17000 blakey
18000 boilers
19000 bpm
20000 bronze
21000 burgnich
22000 calvary
23000 carpelan
24000 ceirtain
25000 charge
26000 chlorophyta
27000 civillist
28000 coauthor
29000 commeneced
30000 coneheads
31000 contflict
32000 correspondant
33000 creb
34000 cudnt
35000 dainomite
36000 deadend
37000 delamarche
38000 derrier
39000 dialectal
40000 diry
41000 distant
42000 donkeykong
43000 drowning
44000 eachothers
45000 egad
46000 emaneuel
47000 enigmasoldier
48000 ern
49000 euwe
50000 experimental
51000 falcionek
52000 feckless
53000 finalist
54000 flunk
55000 fozzy
56000 ftrl
57000 gallium
58000 generated
59000 gilliam
60000 golat
61000 gratim
62000 guidelinesthe
63000 halab
64000 harvested
65000 helicopterllama
66000 hiineedrequestforcomment
67000 ho

In [19]:
import scipy.sparse as sparse
my_embeddings = sparse.csr_matrix(embeds)

In [20]:
doc_embeds = bag_of_words.dot(my_embeddings)

In [21]:
doc_embeds

<159571x100 sparse matrix of type '<class 'numpy.float64'>'
	with 15941500 stored elements in Compressed Sparse Row format>

In [22]:
X_train_de, X_test_de, y_train_de, y_test_de = train_test_split(doc_embeds, toxic, test_size=0.20, random_state=42)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score

In [29]:
dt = DecisionTreeClassifier(max_depth=30)                      #Generating AUC_SCORE FOR DecisionTree Model
dt=dt.fit(X_train_de,y_train_de)
y_pred_dt = dt.predict_proba(X_test_de)
y_pred_dt = np.transpose([pred[:, 1] for pred in y_pred_dt])
t= roc_auc_score(y_test_de, y_pred_dt, average='weighted')
print(t)

0.6166781374646301


In [27]:
rf = RandomForestClassifier(max_depth=10, n_estimators=100)        #Generating AUC_SCORE FOR RandomForest Model
rf=rf.fit(X_train_de,y_train_de)
y_pred_rf = rf.predict_proba(X_test_de)
y_pred_rf = np.transpose([pred[:, 1] for pred in y_pred_rf])
c=roc_auc_score(y_test_de, y_pred_rf, average='weighted')

In [28]:
print(c)

0.9208097120287982


In [30]:
a=[30,40,50,60,70]                                             #Hyperparameter Tunning for decision tree
s=[]                     
for i in a: 
  dt = DecisionTreeClassifier(max_depth=i)
  dt=dt.fit(X_train_de,y_train_de)
  y_pred_dt = dt.predict_proba(X_test_de)
  y_pred_dt = np.transpose([pred[:, 1] for pred in y_pred_dt])
  roc_auc_score(y_test_de, y_pred_dt, average='weighted')
  s.append(t)

In [ ]:
import matplotlib.pyplot as plt                                #Plotting of obtained roc score with hyper parameters
plt.plot(a,s)
plt.xlabel("HYPERPARAMETER : max_depth")
plt.ylabel("AUC SCORE")
plt.show

In [ ]:
a=[10,20,30,40]                                              #Hyperparameter Tunning Random Forest Classification
b=[25,50,75,100] 
k=[]
for i in a:
  for j in b:
    rf = RandomForestClassifier(max_depth=a, n_estimators=b)
    rf=rf.fit(X_train_de,y_train_de)
    y_pred_rf = rf.predict_proba(X_test_de)
    y_pred_rf = np.transpose([pred[:, 1] for pred in y_pred_rf])
    c=roc_auc_score(y_test_de, y_pred_rf, average='weighted')
    k.append(c)

In [ ]:
plt.plot(k,a)                                          #Plotting of obtained roc score with hyper parameter max_depth
plt.show

In [ ]:
plt.plot(k,b)                                          #Plotting of obtained roc score with hyper parameter n_estimators
plt.show